In [1]:
import random
import copy
import time

In [2]:
# dodajemo sve grane koje vode ka neposecenim cvorovima
def add_neighbours(visited, neighbours, graph):
    for node in visited:
        for edge in graph[node]:
            if edge not in visited:
                neighbours.append((node, edge))

In [3]:
def traverse(i, graph, alpha, beta, k):
    n = len(graph)
    tree = []
    tree_weight = 0
    visited = [i]
    while len(visited) < k:
        neighbours = []
        add_neighbours(visited, neighbours, graph)
        if len(neighbours) == 0:
            break
        values = [graph[v[0]][v[1]]['pheromones']**alpha / graph[v[0]][v[1]]['weight']**beta for v in neighbours]
        chosen_neighbour = random.choices(neighbours, weights = values, k = 1)[0]
        tree.append(chosen_neighbour)
        tree_weight += graph[chosen_neighbour[0]][chosen_neighbour[1]]['weight']
        visited.append(chosen_neighbour[1])
    
    if len(visited) != k:
        return ([], float('inf'))
    
    return tree, tree_weight

In [5]:
def calculate_convergence_factor(graph, best_tree, k):
    pheromone_list = [graph.edges[edge]['pheromones'] for edge in best_tree[0]]
    pheromone_sum = sum(pheromone_list)
    
    return pheromone_sum / k * 0.99

In [6]:
def aco(graph_orig, num_ants, rho, num_iters, theta, alpha, beta, k):
    start = time.time()
    graph = copy.deepcopy(graph_orig)
    n = len(graph_orig)
    best_tree = ([], float('inf'))
    
    for i in range(num_iters):
        node_best_tree = ([], float('inf'))
        
        chosen_nodes = random.choices(range(n), k = num_ants)
        trees = [traverse(i, graph, alpha, beta, k) for i in chosen_nodes]

        for edge in graph.edges:
            graph[edge[0]][edge[1]]['pheromones'] *= rho

        for tree, tree_weight in trees:
            delta = theta / tree_weight
            for branch in tree:
                graph[branch[0]][branch[1]]['pheromones'] += delta

        ants_best_tree = min(trees, key = lambda c: c[1])
        if node_best_tree[0] == [] or ants_best_tree[1] < node_best_tree[1]:
            node_best_tree = copy.deepcopy(ants_best_tree)

        if best_tree[0] == [] or node_best_tree[1] < best_tree[1]:
            best_tree = copy.deepcopy(node_best_tree)
            
        cf = calculate_convergence_factor(graph, best_tree, k)
        if cf >= 0.99:
            graph = copy.deepcopy(graph_orig)

    if best_tree[0] != []:
        print(best_tree[0], best_tree[1])
    else:
        print("Not possible to construct a sufficient tree")
        
    end = time.time()
    print(end - start)
    
    return best_tree